<a href="https://colab.research.google.com/github/diegoramzam12/CRUD_PYTHON/blob/main/CRUD_Distribuidores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lista de entregables**

[Accesos a db remota](https://drive.google.com/file/d/1MRvWKGpY80PkMxblVv52MT6qCOSYtsUC/view?usp=sharing)

[Script para creación de datos](https://drive.google.com/file/d/1GcDZtLgVy-KrkcUnlgzVc1NAjqJc6Scf/view?usp=sharing)

[Archivo de consultas mysql](https://drive.google.com/file/d/1JFjTowOZfU97le3wCuXaXd9vHc9_K2l1/view?usp=sharing)

[Repositorio|Github](https://)



In [ ]:
#INSTALACION DE DEPENDENCIAS PARA LA CONEXION A DB
#!sudo apt-get install python3-dev default-libmysqlclient-dev
#!pip install pymysql
#!pip install mysql-connector-python
#!pip install sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#LIBRERIAS
#Para limpiar pantalla
from IPython.display import clear_output
#Para los numeros random
from random import seed
from random import randint
#Para las pausas
from time import sleep
#Para conectar con mysql
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import func
#Para crear modelos de db
from sqlalchemy.sql.functions import now
from sqlalchemy import Column, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
#from sqlalchemy.orm import sessionmaker
#Para ejecutar queries
from sqlalchemy import text
from sqlalchemy import insert, update, delete


In [ ]:
#MODELOS
base = declarative_base()
class distributors(base):
  __tablename__ = "distributors"
  id_distributor = Column(Integer,primary_key=True)
  dist_id = Column(String)
  date_sign = Column(server_default=now(), nullable=False)

class persons(base):
  __tablename__ = "persons"
  id_person = Column(Integer,primary_key=True)
  distributor_id = Column(String)
  name_p = Column(String)
  f_last_name = Column(String)
  m_last_name = Column(String)

class adresses(base):
  __tablename__ = "adresses"
  id_adress = Column(Integer,primary_key=True)
  distributor_id = Column(String)
  st_name = Column(String)
  st_number = Column(Integer)
  nbh_name = Column(String)

class phone_numbers(base):
  __tablename__ = "phone_numbers"
  id_phone_number = Column(Integer,primary_key=True)
  distributor_id = Column(String)
  ph_number = Column(String)
  isactive = Column(Boolean)

In [ ]:
#CONEXION A DB REMOTA

#Conectando a db
MYSQL_HOSTNAME = 'sql3.freemysqlhosting.net'
MYSQL_USER = 'sql3500511'
MYSQL_PASSWORD = 'ufc8mVAas1'
MYSQL_DATABASE = 'sql3500511'
connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}/{MYSQL_DATABASE}'
engine = create_engine(connection_string)

In [ ]:
#Opciones del menu
menu_options = {
    1: 'Alta distribuidores',
    2: 'Actualizar distribuidor',
    3: 'Eliminar distribuidor',
    4: 'Listar',
    5: 'Buscar por id',
    6: 'Asingar nuevo telefono a distribuidor',
    7: 'Salir'
}
#Mostrar menu
def print_menu():
    clear_output()
    for key in menu_options.keys():
        print (key, '--', menu_options[key] )
#Generar id distribuidor
def generate_id():
  valores = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J']
  start = 0
  end = 19
  id = valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]+valores[randint(start, end)]
  return id
#Funciones del menu
#ALTA
def option1():
     clear_output()
     distID = generate_id()
     print("-----CAPTURE LOS DATOS DEL DISTRIBUIDOR-----")
     print("ID DIST: ",distID)
     sleep(.2)
     nomDist = input("Nombre(s):")
     sleep(.2)
     apPatDist = input("Apellido paterno:")
     sleep(.2)
     apMatDist = input("Apellido materno:")
     sleep(.2)
     dirCalleDist = input("Calle:")
     sleep(.2)
     dirNumDist = input("Numero Exterior:")
     sleep(.2)
     dirColDist = input("Colonia:")
     sleep(.2)
     telDist = input("Telefono:")
     print("--------------------------------------------")

    #Insert queries
     stmt_d = insert(distributors).values(dist_id=distID)
     stmt_p = insert(persons).values(distributor_id=distID,name_p=nomDist,f_last_name=apPatDist,m_last_name=apMatDist)
     stmt_ad = insert(adresses).values(distributor_id=distID,st_name=dirCalleDist,st_number=dirNumDist,nbh_name=dirColDist)
     stmt_pn = insert(phone_numbers).values(distributor_id=distID,ph_number=telDist)
     with engine.connect() as conn:
       try:
        result = conn.execute(stmt_d)
        result = conn.execute(stmt_p)
        result = conn.execute(stmt_ad)
        result = conn.execute(stmt_pn)

        print("Datos guardados")   
       except SQLAlchemyError as e:
        error = str(e.__dict__['orig'])
        print("Ocurrio un error al insertar en la tabla ",error)
     sleep(.5)
     input("*Presione 'Enter' para continuar")

#ACTUALIZAR
def option2():
     clear_output()
     distID = input("Ingrese el ID del colaborador a Actualizar:")
     query = "SELECT d.dist_id id, p.name_p nom, p.f_last_name apat, p.m_last_name amat,pn.ph_number tel, ad.st_name calle, ad.st_number num_ext, ad.nbh_name col from distributors d inner join persons p on p.distributor_id = d.dist_id inner join adresses ad on ad.distributor_id = d.dist_id inner join phone_numbers pn on pn.distributor_id = d.dist_id where d.dist_id ='"+distID+"'" 
     encontrado = False
     #Buscando distribudor
     with engine.connect() as connection:
        result = connection.execute(text(query))
        for row in result:
          dist_info = row
          encontrado = True
        sleep(.5)
        if not encontrado :
          input("El id no existe o es incorrecto, presione 'Enter' para continuar")
          return 0
        else :
          #Capturando campos a actualizar
          distID = dist_info['id'];
          print("-----ACTUALICE LOS DATOS DEL DISTRIBUIDOR-----")
          print("[DEJE EN BLANCO PARA NO ACTUALIZAR]")
          print("ID DIST: ",distID)
          sleep(.2)
          nomDist = input("Nombre(s):")
          sleep(.2)
          apPatDist = input("Apellido paterno:")
          sleep(.2)
          apMatDist = input("Apellido materno:")
          sleep(.2)
          dirCalleDist = input("Calle:")
          sleep(.2)
          dirNumDist = input("Numero Exterior:")
          sleep(.2)
          dirColDist = input("Colonia:")
          sleep(.2)
          telDist = input("Telefono:")
          print("--------------------------------------------")
          #Validando campos sin cambios
          if not nomDist :
            nomDist = dist_info['nom']
          if not apPatDist :
            apPatDist = dist_info['apat']
          if not apMatDist :
            apMatDist = dist_info['amat']
          if not dirCalleDist :
            dirCalleDist = dist_info['calle']
          if not dirNumDist :
            dirNumDist = dist_info['num_ext']
          if not dirColDist :
            dirColDist = dist_info['col']
          if not telDist :
            telDist = dist_info['tel']
          #Update queries
          stmt_p = update(persons).where(persons.distributor_id == distID).values(name_p=nomDist,f_last_name=apPatDist,m_last_name=apMatDist)
          stmt_ad = update(adresses).where(adresses.distributor_id == distID).values(st_name=dirCalleDist,st_number=dirNumDist,nbh_name=dirColDist)
          stmt_pn = update(phone_numbers).where(phone_numbers.distributor_id == distID).values(ph_number=telDist)
          with engine.connect() as conn:
            try:
              result = conn.execute(stmt_p)
              result = conn.execute(stmt_ad)
              result = conn.execute(stmt_pn)

              print("Datos actualizados")   
            except SQLAlchemyError as e:
              error = str(e.__dict__['orig'])
              print("Ocurrio un error al actualizar la tabla ",error)
        sleep(.5)
        input("Presione 'Enter' para continuar ")

#ELIMINAR
def option3():
      clear_output()
      distID = input("Ingrese el ID del colaborador a Eliminar:")
      query = 'select d.dist_id ID,CONCAT(p.name_p," ",p.f_last_name," ", p.m_last_name) NOMBRE_DIST, CONCAT("CALLE ",ad.st_name," #", ad.st_number,", ", ad.nbh_name) DIRECCION_DIST, pn.ph_number TEL_DIST from distributors d inner join persons p on p.distributor_id = d.dist_id inner join adresses ad on ad.distributor_id = d.dist_id inner join phone_numbers pn on pn.distributor_id = d.dist_id where pn.isactive = true and d.dist_id = "'+distID+'"'
      encontrado = False
      #Buscando distribudor
      with engine.connect() as connection:
          result = connection.execute(text(query))
          for row in result:
            dist_info = row
            encontrado = True
          sleep(.5)
          if not encontrado :
            input("El id no existe o es incorrecto, presione 'Enter' para continuar")
            return 0
          else :
           stmt_d = delete(distributors).where(distributors.dist_id == distID)
           stmt_p = delete(persons).where(persons.distributor_id == distID)
           stmt_ad = delete(adresses).where(adresses.distributor_id == distID)
           stmt_pn = delete(phone_numbers).where(phone_numbers.distributor_id == distID)
          with engine.connect() as conn:
            try:
              result = conn.execute(stmt_d)
              result = conn.execute(stmt_p)
              result = conn.execute(stmt_ad)
              result = conn.execute(stmt_pn)

              print("El distribuidor fue eliminado exitosamente")   
            except SQLAlchemyError as e:
              error = str(e.__dict__['orig'])
              print("Ocurrio un error al actualizar la tabla ",error)
          sleep(.5)
          input("Presione 'Enter' para continuar ")
#LISTAR
def option4():
    clear_output()
    #consulta
    query = 'select d.dist_id ID,CONCAT(p.name_p," ",p.f_last_name," ", p.m_last_name) NOMBRE_DIST, CONCAT("CALLE ",ad.st_name," #", ad.st_number,", ", ad.nbh_name) DIRECCION_DIST, pn.ph_number TEL_DIST from distributors d inner join persons p on p.distributor_id = d.dist_id inner join adresses ad on ad.distributor_id = d.dist_id inner join phone_numbers pn on pn.distributor_id = d.dist_id where pn.isactive = true'
    with engine.connect() as connection:
        result = connection.execute(text(query))
        for row in result:
            print("-----------------------------------------")
            print("ID:", row['ID'])
            print("NOMBRE COMPLETO:", row['NOMBRE_DIST'])
            print("DIRECION:", row['DIRECCION_DIST'])
            print("TEL:", row['TEL_DIST'])
        input("Presione 'Enter' para continuar")
#BUSCAR
def option5():
  clear_output()
  distID = input("Ingrese el ID del colaborador a Buscar:")
  query = 'select d.dist_id ID,CONCAT(p.name_p," ",p.f_last_name," ", p.m_last_name) NOMBRE_DIST, CONCAT("CALLE ",ad.st_name," #", ad.st_number,", ", ad.nbh_name) DIRECCION_DIST, pn.ph_number TEL_DIST from distributors d inner join persons p on p.distributor_id = d.dist_id inner join adresses ad on ad.distributor_id = d.dist_id inner join phone_numbers pn on pn.distributor_id = d.dist_id where pn.isactive = true and d.dist_id = "'+distID+'"'
  encontrado = False
  #Buscando distribudor
  with engine.connect() as connection:
      result = connection.execute(text(query))
      for row in result:
        dist_info = row
        encontrado = True
      sleep(.5)
      if not encontrado :
        input("El id no existe o es incorrecto, presione 'Enter' para continuar")
        return 0
      else :
        with engine.connect() as connection:
          result = connection.execute(text(query))
          for row in result:
              print("-----------------------------------------")
              print("ID:", row['ID'])
              print("NOMBRE COMPLETO:", row['NOMBRE_DIST'])
              print("DIRECION:", row['DIRECCION_DIST'])
              print("TEL:", row['TEL_DIST'])
          input("Presione 'Enter' para continuar")
#ASIGNACION DE TEL
def option6():
  clear_output()
  distID = input("Ingrese el ID del colaborador a quien se asignara el nuevo telefono:")
  query = 'select d.dist_id ID,CONCAT(p.name_p," ",p.f_last_name," ", p.m_last_name) NOMBRE_DIST, CONCAT("CALLE ",ad.st_name," #", ad.st_number,", ", ad.nbh_name) DIRECCION_DIST, pn.ph_number TEL_DIST from distributors d inner join persons p on p.distributor_id = d.dist_id inner join adresses ad on ad.distributor_id = d.dist_id inner join phone_numbers pn on pn.distributor_id = d.dist_id where pn.isactive = true and d.dist_id = "'+distID+'"'
  encontrado = False
  #Buscando distribudor
  with engine.connect() as connection:
      result = connection.execute(text(query))
      for row in result:
        dist_info = row
        encontrado = True
      sleep(.5)
      if not encontrado :
        input("El id no existe o es incorrecto, presione 'Enter' para continuar")
        return 0
      else :
        with engine.connect() as connection:
          result = connection.execute(text(query))
          for row in result:
              print("-----------------------------------------")
              print("ID:", row['ID'])
              print("NOMBRE COMPLETO:", row['NOMBRE_DIST'])
              print("DIRECION:", row['DIRECCION_DIST'])
              print("TEL:", row['TEL_DIST'])
          tel = input("Por favor ingrese el nuevo numero de telefono[el numero de telefono actual sera deshabilitado]: ")
          if not tel:
            tel = dist_info['TEL_DIST']
          stmt_pn_upd = update(phone_numbers).where(phone_numbers.distributor_id == distID).values( isactive = False);
          stmt_pn_in = insert(phone_numbers).values(ph_number=tel,distributor_id = distID)
          with engine.connect() as conn:
            try:
              result = conn.execute(stmt_pn_upd)
              result = conn.execute(stmt_pn_in)

              print("El numero de telefono se actualizo correctamente")   
            except SQLAlchemyError as e:
              error = str(e.__dict__['orig'])
              print("Ocurrio un error al actualizar la tabla ",error)
          sleep(.5)
          input("Presione 'Enter' para continuar")

In [ ]:
#CRUD
if __name__=='__main__':
    while(True):
        print_menu()
        option = ''
        sleep(.5)
        try:
            option = int(input('Por favor elija una opcion: '))
        except:
            print('Valor invalido. Por favor ingrese un valor numerico ...')
        #Revisar que opcion se eligio y llamar el respectivo metodo
        if option == 1:
           option1()
        elif option == 2:
            option2()
        elif option == 3:
            option3()
        elif option == 4:
            option4()
        elif option == 5:
            option5()
        elif option == 6:
            option6()
        elif option == 7:
            clear_output()
            sleep(.5)
            print('Tarea terminada')
            break
        else:
            input('Opcion invalida. por favor ingrese un numero entre 1 y 7. Presione "Enter" para continuar')

Tarea terminada
